In [3]:
import os
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import scipy.stats as st
from scipy.stats import linregress
from matplotlib import style
import matplotlib.pyplot as plt
from sklearn import datasets
from jupyterthemes import jtplot
import json

from datetime import datetime

In [5]:
stock_name = "resources/stocks_list.csv"
stockname_df = pd.read_csv(stock_name)
# stockname_df.head(10)

In [ ]:
# stockname_df.to_html("stockname.html",index = False, classes = ["table", "table-hover", "table-striped"])

In [6]:
# Extract the data set from csv file: 
stock = "resources/all_stocks_5yr.csv"
allstock_df = pd.read_csv(stock)
#rename column for merge
allstock_df=allstock_df.rename(columns={"Name":"Symbol"})
# allstock_df.head(10)

In [8]:
#load helper csv to add name and industry to original dataset
stockname="resources/stocks_list.csv"
stockname_df=pd.read_csv(stockname)
stockname_df=stockname_df[['Symbol','Name','Industry','Market Cap']]
# stockname_df.head()

In [9]:
#merge 2 datasets
merged_df=pd.merge(allstock_df ,stockname_df, how='left', on='Symbol')
# merged_df.head()

In [10]:
# narrow down into 1 year (2018) 
one_year_df = merged_df.loc[merged_df["date"] >= "2017-01-01"]
# one_year_df

In [11]:
one_year_df = one_year_df.reset_index(drop=True)
# one_year_df

In [12]:
quantity = one_year_df["Name"].unique()
# quantity

In [13]:
#View all Industry in the orignal dataset to select which industry to focus on
one_year_df["Industry"].unique()

array(['Airlines', 'Technology Hardware, Storage & Periph...',
       'Specialty Retail', 'Biotechnology',
       'Health Care Providers & Services',
       'Health Care Equipment & Supplies', 'IT Services', 'Software',
       'Semiconductors & Semiconductor Equipment', 'Food Products',
       'Multi-Utilities', 'Electric Utilities',
       'Independent Power and Renewable Elect...', nan, 'Insurance',
       'Equity Real Estate Investment Trusts ...', 'Chemicals',
       'Building Products', 'Electrical Equipment', 'Capital Markets',
       'Internet & Direct Marketing Retail',
       'Oil, Gas & Consumable Fuels',
       'Electronic Equipment, Instruments & C...', 'Auto Components',
       'Aerospace & Defense', 'Entertainment', 'Containers & Packaging',
       'Water Utilities', 'Consumer Finance',
       'Life Sciences Tools & Services', 'Banks', 'Beverages',
       'Pharmaceuticals', 'Diversified Financial Services', 'Machinery',
       'Hotels, Restaurants & Leisure', 'Health Care

In [ ]:
#Dataframe with Pharmaceutical industries into json file
pharm_df=one_year_df.loc[one_year_df["Industry"] == "Pharmaceuticals"]
pharm_df["Name"].unique()

In [ ]:
#Dataframe with "Technology Hardware, Storage & Periph..." industries into json file
tech_df=one_year_df.loc[one_year_df["Industry"] == "Technology Hardware, Storage & Periph..."]
tech_df["Name"].unique()
#tech_df.to_json(r'jsonfiles/technology.json')

In [ ]:
#Dataframe with Banks industries into json file
banks_df=one_year_df.loc[one_year_df["Industry"] == "Banks"]
#banks_df.to_json(r'jsonfiles/finance.json')
banks_df

In [ ]:
BanksQuantity = banks_df["Name"].unique()
BanksQuantity

In [ ]:
# extract the close prices our to a list for boxplot to
BAC = banks_df.loc[banks_df["Name"]=="Bank of America","close"]
CFG = banks_df.loc[banks_df["Name"]=="Citizens Financial Group","close"]
COM = banks_df.loc[banks_df["Name"]== "Comerica","close"]
CITI = banks_df.loc[banks_df["Name"]== "Citigroup","close"]
FifThird = banks_df.loc[banks_df["Name"]== "Fifth Third Bancorp","close"]
HBancshares = banks_df.loc[banks_df["Name"]== "Huntington Bancshares","close"]


In [ ]:
bankStock_list = ["Bank of America", "Citizens Financial Group", "Comerica", "Citigroup","Fifth Third Bancorp","Huntington Bancshares"]

# Create empty list to fill with stock close prices
bank_price_list = []

# Calculate the IQR and quantitatively determine if there are any potential outliers. 
for price in bankStock_list:
    
    
    prices = banks_df.loc[banks_df["Name"] == price, 'close']
    
    # add subset 
    bank_price_list.append(prices)
    
    # Determine outliers using upper and lower bounds
    quartiles = prices.quantile([.25,.5,.75])
    print(quartiles)

In [ ]:
quartiles = BAC.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq

print(f"The lower quartile of bank stock prices in the year is: {lowerq}")
print(f"The upper quartile of bank stock prices in the year is: {upperq}")
print(f"The interquartile range of bank stock prices in the year is: {iqr}")
print(f"The the median of bank stock prices in the year is: {quartiles[0.5]} ")

lower_bound = lowerq - (1.5*iqr)  # lower the lower (1.5 times the iqr)|upper the upper () -> ouliers 
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

In [ ]:
# boxplot:

fig1, ax1 = plt.subplots()

plt.boxplot(bank_price_list,labels=bankStock_list)

In [ ]:
# convert 3 df into 3 csv files: 

pharm_df.to_csv("pharm.csv", index = False)

In [ ]:
tech_df.to_csv("tech.csv", index = False)

In [ ]:
banks_df.to_csv("bank.csv", index = False)

In [ ]:
# manipulating and put into SQL database:

# connect to local database: 
connection = "postgres:Postgres@localhost:5432/stockPandas_db"
engine = create_engine(f'postgresql://{connection}')

# put df into sql database format:

#allstock_df.to_sql(name='stock_master', con=engine, if_exists='append', index=False)
banks_df.to_sql(name='stock_banks', con = engine,if_exists='append',index=False)
tech_df.to_sql(name='stock_tech', con = engine,if_exists='append',index=False)
pharm_df.to_sql(name='stock_pharm', con = engine,if_exists='append',index=False)

In [ ]:
# Consolidate data to 3 types of bank and 1 month period
bank1mo = banks_df.loc[(banks_df["Name"]=="Bank of America") | (banks_df["Name"]=="JPMorgan Chase & Co.") | (banks_df["Name"]=="Wells Fargo & Company"), :]
bank1mo=bank1mo.loc[bank1mo["date"] > "2018-01-06"]
bank1mo.to_csv("bank1mo.csv",index=False)

In [ ]:
# Consolidate data to 3 types of bank and 1 month period
pharm1mo = pharm_df.loc[(pharm_df["Name"]=="Johnson & Johnson") | (pharm_df["Name"]=="Merck & Co.") | (pharm_df["Name"]=="Pfizer"), :]
pharm1mo=pharm1mo.loc[pharm1mo["date"] > "2018-01-06"]

pharm1mo.to_csv("pharm1mo.csv",index=False)

In [ ]:
# Consolidate data to 3 types of bank and 1 month period
tech1mo = tech_df.loc[(tech_df["Name"]=="Apple") | (tech_df["Name"]=="Seagate Technology Holdings") | (tech_df["Name"]=="Western Digital"), :]
tech1mo=tech1mo.loc[tech1mo["date"] > "2018-01-06"]
tech1mo.to_csv("tech1mo.csv",index=False)